# Economic Growth over years vs. Exit of Airlines


Some decisions that I made here:

- I am doing a syntheic control model
- I am going to only consider data in or beyond 2003 for two reasons:
1) 9/11 creates an artificial jump in airline route cancellations
2) the data that I have starts in 2001 so I don't have enough data to do comparables analysis if I start too early

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import k-means for analysis while doing EDA
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

# the package that is going to provide us with our DID method
# for creating a synthetic control model

# we are going to be implementing a difference in differences with synthetic controls method
from pydid import DID

In [ ]:
### SETTINGS

min_pass_per_day = 100
min_ms = 0.7                # not helpful currently


### PROBABLY DON'T TOUCH

path_to_aggregated_weather_data = "/Users/tristanbrigham/Desktop/Citadel Datathon/Local Important Data/Auxillary Data/NOAA Disaster Data/final_weather.csv"
news_weather_df = pd.read_csv(path_to_aggregated_weather_data)

path_to_economic_growth_fips = "/Users/tristanbrigham/Desktop/Citadel Datathon/Local Important Data/Auxillary Data/FIPS econ data/us_county_latlng.csv"
fips_mapping_df = pd.read_csv(path_to_economic_growth_fips)

path_to_county_growth = "/Users/tristanbrigham/Desktop/Citadel Datathon/Local Important Data/CAGDP9/CAGDP9__ALL_AREAS_2001_2021.csv"
econ_growth_df = pd.read_csv(path_to_county_growth, encoding='latin-1')

path_to_market_id_mapping = "/Users/tristanbrigham/Desktop/Citadel Datathon/Citadel_Correlation_One_Datathon/crucial_data_and_files/code_to_region_state_mapping.csv"
market_id_mapping_df = pd.read_csv(path_to_market_id_mapping)

# only going to consider routes that have shut down after at least 4 quarters of operation
path_to_shutdown_data = "/Users/tristanbrigham/Desktop/basic_cancelled_final copy.csv"
shutdown_df = pd.read_csv(path_to_shutdown_data)
shutdown_df = shutdown_df[shutdown_df["amt_quarters_past"] >= 4]

markets_amt_ppl = "/Users/tristanbrigham/Desktop/Citadel Datathon/Citadel_Correlation_One_Datathon/crucial_data_and_files/markets_to_people_per_day.csv"
markets_amt_ppl_df = pd.read_csv(markets_amt_ppl)

alpha = "abcdefghij"
        #0123456789


# this function gets all of the fips codes that are affected by an air route stopping
# it takes the market_id and looks for every fips code (county) which is in that market_id according to the mapping that we generated
def get_affected_fips(fips_1, fips_2):

    ret_arr = []

    [ret_arr.append(str(x)) for x in fips_mapping_df[fips_mapping_df["mkt_id"] == fips_1]["fips_code"].to_numpy()]
    [ret_arr.append(str(x)) for x in fips_mapping_df[fips_mapping_df["mkt_id"] == fips_2]["fips_code"].to_numpy()]

    return ret_arr


# this returns an alphabetical code which is analogous to the fips code
# this gets around the issue of leading zeros in the fips code which makes it difficult for pandas to parse and find
# the correct codes when we request them
def custom_fips_code(fips_code):

    ret_str = ""

    fips_code = int(str(fips_code).replace('"', ''))

    if fips_code == 0:

        return "aaaaa"

    if fips_code < 10000:

        ret_str += "a"

    for digit in str(fips_code):

        ret_str += alpha[int(digit)]
    
    return ret_str



# apply the new code to the dataframe
econ_growth_df["cust_id"] = econ_growth_df["GeoFIPS"].apply(custom_fips_code)
econ_growth_df

In [ ]:
# now we are going to go through and make sure that every FIPS number has an appropriate mapping
# we are going to go through each of the shutdown routes and see what we can find

# get all of the fips that were affected by this shutdown
shutdown_df["fips_affected"] = shutdown_df.apply(lambda row: get_affected_fips(row["citymarketid_1"], row["citymarketid_2"]), axis=1)

shutdown_df = shutdown_df[shutdown_df["Year"] > 2002]
shutdown_df = shutdown_df[shutdown_df["passengers"] > min_pass_per_day]

print(f"We have {len(shutdown_df)} records that we are going to consider")
print(f"VAL CTS YEAR: {shutdown_df['Year'].value_counts()}")
# print(f"VAL CTS YEAR: {shutdown_df['Year']}")


In [ ]:
#### GRAPHS MODELING TOTAL GDP AFTER ROUTE SHUTDOWN


# go through and get the economic data now for each FIPS region and plot it
# we have to get every industry and plot it
# also have to make sure that we are getting every fips area
for idx, row in shutdown_df.iterrows():

    # get the FIPS
    fips_idx = row["fips_affected"]

    # for each of the fips idx
    for code in fips_idx:

        year_center = row["Year"]

        year_min = year_center - 4
        year_max = year_center + 5

        if year_min < 2001:
            year_min = 2001
        
        if year_max > 2021:
            year_max = 2021


        get_years = list(range(year_min, year_max))


        temp_years = []
        for year in get_years:

            if year <= 2021 and year >= 2001:
                temp_years.append(str(year))


        # labels = temp_years
        # labels.append("GeoName")
        # labels.append("GeoFIPS")
        # labels.append("cust_id")
        # labels.append("Description")

        temp_fips = econ_growth_df[econ_growth_df["cust_id"] == custom_fips_code(code)]
        # temp_fips = temp_fips[temp_fips["Description"] == "All industry total "]
        temp_fips = temp_fips[temp_years]

        # # create the datsets
        # index_of_age_column = df.columns.get_loc('Age')


        for _, sub_row in temp_fips.iterrows():

            try:
                plt.plot(get_years, [int(sub_row[lab]) for lab in temp_fips], label=sub_row)
                # print([sub_row[lab] for lab in temp_fips])
            except:
                pass
        
        plt.axvline(x=year_center, color='red', linestyle='--')


    # Add labels, title, and legend
    plt.xlabel('Year')
    plt.ylabel('Industry Value')
    plt.title('GDP by Year')
    # plt.legend(title='Lines:', loc='upper left', frameon=False, prop={'size': 10})

    plt.yscale('linear')

    # Show the plot
    plt.show()

        

    


In [ ]:
temp_fips

In [ ]:
#### GRAPHS MODELING CHANGE IN GDP AFTER A ROUTE SHUTDOWN


# go through and get the economic data now for each FIPS region and plot it
# we have to get every industry and plot it
# also have to make sure that we are getting every fips area
# do percentage change now

temp_shutdown_df = shutdown_df

for idx, row in temp_shutdown_df.iterrows():

    # get the FIPS
    fips_idx = row["fips_affected"]

    # for each of the fips idx
    for code in fips_idx:

        year_center = row["Year"]

        year_min = year_center - 4
        year_max = year_center + 8

        if year_min < 2001:
            year_min = 2001
        
        if year_max > 2021:
            year_max = 2021


        get_years = list(range(year_min, year_max))


        temp_years = []
        for year in get_years:

            if year <= 2021 and year >= 2001:
                temp_years.append(str(year))


        # labels = temp_years
        # labels.append("GeoName")
        # labels.append("GeoFIPS")
        # labels.append("cust_id")
        # temp_years.append("Description")

        temp_fips = econ_growth_df[econ_growth_df["cust_id"] == custom_fips_code(code)].replace('(D)', 0)
        temp_fips = temp_fips[temp_fips["Description"] == "All industry total "]
        
        desc_df = temp_fips["Description"]
        temp_fips = temp_fips[temp_years]

        temp_fips = temp_fips.apply(pd.to_numeric, errors='coerce')
        temp_fips = temp_fips.pct_change(axis=1).drop(columns=temp_years[0])

        # print(desc_df)


    #     # create the datsets
    #     ###### index_of_age_column = df.columns.get_loc('Age')

        for( _, sub_row), (label) in zip(temp_fips.iterrows(), desc_df):

            # print(label)
            # print(len(temp_years[1 : ]))
            # print(len([sub_row[str(lab)] for lab in temp_years[1 :]]))

            try:
                # plt.plot(temp_years[1 : ], [sub_row[str(lab)] for lab in temp_years[1 :]], label=str(label))
                y = [sub_row[str(lab)] if sub_row[str(lab)] < 2 else 2 for lab in temp_years[1 :]]

                plt.plot(temp_years[1 : ], y)
                # print([sub_row[lab] for lab in temp_fips])

            except:
                pass
        
        plt.axvline(x=str(year_center), color='red', linestyle='--')


    # Add labels, title, and legend
    plt.xlabel('Year')
    plt.ylabel('Industry Value Percent Change')
    plt.title('GDP Percent Change by Year')
    plt.legend(title='Lines:', loc='upper left', frameon=False, prop={'size': 4})

    plt.yscale('linear')

    # Show the plot
    plt.show()

        

    


In [ ]:
# go through and get the average gdp change percentage-wise per industry per year
econ_growth_df["Description"] = econ_growth_df["Description"].str.strip()
# econ_growth_df.apply(lambda row: print(row.loc[:, 5]))

temp_gdp_df = econ_growth_df.copy().fillna('(D)') 

custom_string = '(NA)'
temp_gdp_df = temp_gdp_df[~temp_gdp_df.apply(lambda row: custom_string in row.values, axis=1)]


labels = set(temp_gdp_df["Description"])
new_labels = set([l.strip() for l in labels])

avg_change_year_dict = {}

for label in new_labels:

    curr_df = temp_gdp_df[temp_gdp_df["Description"] == label]

    label_avg_dict = {}

    # go through and get each percentage change
    for idx, row in curr_df.iterrows():

        cols = [str(yr) for yr in list(range(2001, 2022))]
        
        values = row[cols]

        for col_id, col_name in enumerate(cols[:-1]):

            if row[cols[col_id]] != '(D)' and row[cols[col_id + 1]] != '(D)' and int(row[cols[col_id]]) != 0:

                if col_name in label_avg_dict:
                    label_avg_dict[col_name].append((int(row[cols[col_id + 1]]) / int(row[cols[col_id]])) - 1)
                else:
                    label_avg_dict[col_name] = [(int(row[cols[col_id + 1]]) / int(row[cols[col_id]])) - 1]
    
    avg_change_year_dict[label] = label_avg_dict


In [ ]:
for label in new_labels:

    for year in list(range(2001, 2021)):

        try:
            avg_change_year_dict[label][str(year)] = np.mean(avg_change_year_dict[label][str(year)])
        except:
            avg_change_year_dict[label][str(year)] = 0

avg_change_year_dict


In [ ]:
# fill in the dataframe with projected values
cols = [str(yr) for yr in list(range(2001, 2022))]

# go row by row and fill in the missing values
for idx, row in temp_gdp_df.iterrows():

    if idx % 100 == 0:

        print(idx)

    # check if we are missing any values
    if '(D)' in row[cols].values:

        if temp_gdp_df.at[idx, "2001"] != "(D)":
            # first value is not a null

            for col_name in cols:

                if temp_gdp_df.at[idx, col_name] == '(D)':

                    temp_gdp_df.at[idx, col_name] = int(temp_gdp_df.at[idx, str(int(col_name) - 1)]) * (1 + avg_change_year_dict[temp_gdp_df.at[idx, "Description"]][str(int(col_name) - 1)])

        else:
            # the first value is a null

            if row[cols].nunique() == 1:

                for col_name in cols:

                    temp_gdp_df.at[idx, col_name] = 0
            
            else:

                # go through and find the numerical row
                interesting_col = "2001"
                for col_name in cols:

                    if temp_gdp_df.at[idx, col_name] != '(D)':

                        interesting_col = col_name
                        break
                
                _year = int(interesting_col) + 1
                # propogate forwards
                while _year < 2022:

                    if temp_gdp_df.at[idx, str(_year)] == '(D)':


                        temp_gdp_df.at[idx, str(_year)] = int(int(temp_gdp_df.at[idx, str(_year - 1)]) * (1 + avg_change_year_dict[temp_gdp_df.at[idx, "Description"]][str(_year - 1)]))
                    
                    _year += 1


                _year = int(interesting_col) - 1

                # propogate backwards
                while _year > 2000:

                    temp_gdp_df.at[idx, str(_year)] = int(int(temp_gdp_df.at[idx, str(_year + 1)]) / (1 + avg_change_year_dict[temp_gdp_df.at[idx, "Description"]][str(_year)]))
                    _year -= 1

                


# save the new file to csv
temp_gdp_df.to_csv("~/Desktop/non_dropped_gdp_mapping.csv", index=False)

In [ ]:
temp_gdp_df

In [ ]:

# getting statistics on how often each industry category drops after regional flights stop
gdp_delta = {}

labels = []

diff_max = 0

pre_idx_max = -1
post_idx_max = -1

### GO THROUGH TO FIND MAX TOT CHANGE

for diff_pre_set in range(2, 5):

    for diff_post_set in range(2, 8):

        # getting statistics on how often each industry category drops after regional flights stop
        gdp_delta = {}
        labels = []
        avg_arr = []

        for idx, row in temp_shutdown_df.iterrows():

            # get the FIPS
            fips_idx = row["fips_affected"]

            # for each of the fips idx
            for code in fips_idx:

                year_center = row["Year"]

                year_min = year_center - 5
                year_max = year_center + 8

                if year_min < 2001:
                    year_min = 2001
                
                if year_max > 2021:
                    year_max = 2021


                get_years = list(range(year_min, year_max))


                temp_years = []
                for year in get_years:

                    if year <= 2021 and year >= 2001:
                        temp_years.append(str(year))


                temp_fips = temp_gdp_df[temp_gdp_df["cust_id"] == custom_fips_code(code)].replace('(D)', 0)
                # temp_fips = temp_fips[temp_fips["Description"] == "All industry total "]
                
                desc_df = temp_fips["Description"]
                temp_fips = temp_fips[temp_years]

                temp_fips = temp_fips.apply(pd.to_numeric, errors='coerce')
                temp_fips = temp_fips.pct_change(axis=1).drop(columns=temp_years[0]).replace(np.inf, 5).replace(-np.inf, -5).fillna(0) 


                for( _, sub_row), (label) in zip(temp_fips.iterrows(), desc_df):

                    label = label.strip()

                    # get the average growth for the industry in the area
                    diff_pre = int(year_center) - int(temp_years[1])
                    if diff_pre > diff_pre_set:
                        diff_pre = diff_pre_set
                    elif diff_pre <= 0:
                        continue

                    diff_post = int(temp_years[-1]) - int(year_center)
                    if diff_post > diff_post_set:
                        diff_post = diff_post_set
                    elif diff_post <= 0:
                        continue

                    y_pre = sum([sub_row[str(lab)] for lab in list(range(int(year_center) - diff_pre, int(year_center)))])
                    y_post = sum([sub_row[str(lab)] for lab in list(range(int(year_center), int(year_center) + diff_post))])

                    avg_y_pre = y_pre / diff_pre
                    avg_y_post = y_post / diff_post

                    if label in gdp_delta:
                        gdp_delta[label].append([avg_y_pre, avg_y_post, avg_y_post - avg_y_pre])
                    else:
                        gdp_delta[label] = [[avg_y_pre, avg_y_post, avg_y_post - avg_y_pre]]


                    labels.append(label)



        labels = set(labels)

        # print(gdp_delta)

        info_arr = []
        to_csv_dict = {}

        for label in labels:

            temp_arr = gdp_delta[label]

            pre_sum = 0
            post_sum = 0
            diff_sum = 0

            if temp_arr == None:
                continue 

            for t_set in temp_arr:

                pre_sum += t_set[0]
                post_sum += t_set[1]
                diff_sum += t_set[2]
            
            pre_avg = pre_sum / len(temp_arr)
            post_avg = post_sum / len(temp_arr)
            diff_avg = diff_sum / len(temp_arr)

            info_arr.append([label, pre_avg, post_avg, diff_avg])
            avg_arr.append(abs(diff_avg))


        diff_sum = sum(avg_arr)

        print(f"sum: {diff_sum} | pre_idx: {diff_pre_set} | post_idx: {diff_post_set}")

        if diff_sum > diff_max:

            diff_max = diff_sum
            pre_idx_max = diff_pre_set
            post_idx_max = diff_post_set



print(f"MAX: pre_idx_max: {pre_idx_max} | post_idx_max: {post_idx_max}")


In [ ]:

## ACTUAL CODE
# getting statistics on how often each industry category drops after regional flights stop
gdp_delta = {}
labels = []

for idx, row in temp_shutdown_df.iterrows():

    # get the FIPS
    fips_idx = row["fips_affected"]

    # for each of the fips idx
    for code in fips_idx:

        year_center = row["Year"]

        year_min = year_center - 5
        year_max = year_center + 8

        if year_min < 2001:
            year_min = 2001
        
        if year_max > 2021:
            year_max = 2021


        get_years = list(range(year_min, year_max))


        temp_years = []
        for year in get_years:

            if year <= 2021 and year >= 2001:
                temp_years.append(str(year))


        # labels = temp_years
        # labels.append("GeoName")
        # labels.append("GeoFIPS")
        # labels.append("cust_id")
        # temp_years.append("Description")

        temp_fips = temp_gdp_df[temp_gdp_df["cust_id"] == custom_fips_code(code)].replace('(D)', 0)
        # temp_fips = temp_fips[temp_fips["Description"] == "All industry total "]
        
        desc_df = temp_fips["Description"]
        temp_fips = temp_fips[temp_years]

        temp_fips = temp_fips.apply(pd.to_numeric, errors='coerce')
        temp_fips = temp_fips.pct_change(axis=1).drop(columns=temp_years[0]).replace(np.inf, 5).replace(-np.inf, -5).fillna(0) 


        for( _, sub_row), (label) in zip(temp_fips.iterrows(), desc_df):

            label = label.strip()

            # get the average growth for the industry in the area
            diff_pre = int(year_center) - int(temp_years[1])
            if diff_pre > pre_idx_max:
                diff_pre = pre_idx_max
            elif diff_pre <= 0:
                continue

            diff_post = int(temp_years[-1]) - int(year_center)
            if diff_post > post_idx_max:
                diff_post = post_idx_max
            elif diff_post <= 0:
                continue

            y_pre = sum([sub_row[str(lab)] for lab in list(range(int(year_center) - diff_pre, int(year_center)))])
            y_post = sum([sub_row[str(lab)] for lab in list(range(int(year_center), int(year_center) + diff_post))])

            avg_y_pre = y_pre / diff_pre
            avg_y_post = y_post / diff_post

            if label in gdp_delta:
                gdp_delta[label].append([avg_y_pre, avg_y_post, avg_y_post - avg_y_pre])
            else:
                gdp_delta[label] = [[avg_y_pre, avg_y_post, avg_y_post - avg_y_pre]]


            labels.append(label)



labels = set(labels)

# print(gdp_delta)

info_arr = []
to_csv_dict = {}

for label in labels:

    temp_arr = gdp_delta[label]

    pre_sum = 0
    post_sum = 0
    diff_sum = 0

    if temp_arr == None:
        continue 

    for t_set in temp_arr:

        pre_sum += t_set[0]
        post_sum += t_set[1]
        diff_sum += t_set[2]
    
    pre_avg = pre_sum / len(temp_arr)
    post_avg = post_sum / len(temp_arr)
    diff_avg = diff_sum / len(temp_arr)

    info_arr.append([label, pre_avg, post_avg, diff_avg])


info_arr.sort(key=lambda x: x[3])

total_arr = [[], [], [], []]

for label, pre_avg, post_avg, diff_avg in info_arr:

    total_arr[0].append(label)
    total_arr[1].append(pre_avg)
    total_arr[2].append(post_avg)
    total_arr[3].append(diff_avg)

    print(f"{diff_avg} :: \t{label} | {pre_avg}\t{post_avg}")


# to_csv_dict["Industry"] = total_arr[0]
# to_csv_dict["Avg Growth Before"] = total_arr[1]
# to_csv_dict["Avg Drop After"] = total_arr[2]
# to_csv_dict["Avg Difference"] = total_arr[3]

# csv_df = pd.DataFrame(to_csv_dict)
# csv_df.to_csv("~/Desktop/industry_gdp_delta_2.csv", index=False)

In [ ]:
# print(gdp_delta)

info_arr = []
to_csv_dict = {}

for label in labels:

    temp_arr = gdp_delta[label]

    pre_sum = 0
    post_sum = 0
    diff_sum = 0

    if temp_arr == None:
        continue 

    for t_set in temp_arr:

        pre_sum += t_set[0]
        post_sum += t_set[1]
        diff_sum += t_set[2]
    
    pre_avg = pre_sum / len(temp_arr)
    post_avg = post_sum / len(temp_arr)
    diff_avg = diff_sum / len(temp_arr)

    info_arr.append([label, pre_avg, post_avg, diff_avg])

info_arr.sort(key=lambda x: x[3])

total_arr = [[], [], [], []]

for label, pre_avg, post_avg, diff_avg in info_arr:

    total_arr[0].append(label)
    total_arr[1].append(pre_avg)
    total_arr[2].append(post_avg)
    total_arr[3].append(diff_avg)

    print(f"{diff_avg} :: \t{label} | {pre_avg}\t{post_avg}")


to_csv_dict["Industry"] = total_arr[0]
to_csv_dict["Avg Growth Before"] = total_arr[1]
to_csv_dict["Avg Drop After"] = total_arr[2]
to_csv_dict["Avg Difference"] = total_arr[3]

csv_df = pd.DataFrame(to_csv_dict)
# csv_df.to_csv("~/Desktop/industry_gdp_delta_2.csv", index=False)
    

In [ ]:
%pip install pydid


In [ ]:

# for calculating disance to market id lat and lon
from math import sin, cos, sqrt, atan2, radians
from pydid import DID

# first we are going to go ahead and run k-means on the file with all of the economic development 
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances

def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(radians, [lat1, lon1, lat2, lon2])

    # Radius of the Earth in kilometers
    earth_radius_km = 6371.0

    # Haversine formula
    d_lat = lat2_rad - lat1_rad
    d_lon = lon2_rad - lon1_rad

    a = sin(d_lat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(d_lon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate the distance in kilometers
    distance_km = earth_radius_km * c

    return distance_km


# this function is going to sequentially search all of the data that we have and look for a time series
# of growth in industries that is close to the growth that we are trying to find 

gdp_cols = [str(yr) for yr in list(range(2001, 2022))]


affected_fips = []
for idx, row in temp_shutdown_df.iterrows():

    # get the FIPS
    fips_idx = row["fips_affected"]

    for fips_code in fips_idx:

        affected_fips.append(custom_fips_code(fips_code))


# creating the df that will house all of the year over year changes for gdp
percent_change_yoy_df = temp_gdp_df.copy()
percent_change_yoy_df[gdp_cols] = percent_change_yoy_df[gdp_cols].apply(pd.to_numeric)
percentage_change = percent_change_yoy_df[gdp_cols].pct_change(axis=1)
percent_change_yoy_df[gdp_cols] = percentage_change
percent_change_yoy_df.drop(columns="2001") # these will not be helpful for us
percent_change_yoy_df.replace([np.nan, np.inf, -np.inf], 0, inplace=True)
percent_change_yoy_df["route_gone"] = percent_change_yoy_df["cust_id"].apply(lambda x: x in affected_fips)

affected_fips = set(affected_fips)

# this is a mean-squared error approach by hand
# we are considering all flight routes -- even the ones that have also had route attrition
def get_closest_by_growth(fips_id, year_of_impact, first_year = 2001, same_time_frame=True, num_to_find=20):

    # get all of the right characteristics   

    # fill in the dataframe with projected values
    # since the percentage change columns have the percentage change from the last year to the year that the value is in, we are going to consider those columns
    cols = [str(yr) for yr in list(range(first_year + 1, year_of_impact))]

    # get the comparison dataframe
    base_information = percent_change_yoy_df[percent_change_yoy_df["cust_id"] == custom_fips_code(fips_id)][cols].copy()

    # min MSE
    min_mse_val = [float('inf') for i in range(num_to_find)]
    min_mse_fips = [-1 for i in range(num_to_find)]

    # we will iterate through all of the possible fips that we have data on, and calculate the mean squared error for the dataframe values
    set_unique = set(percent_change_yoy_df["cust_id"].unique())
    
    for search_fips in list(set_unique):

        if search_fips == custom_fips_code(fips_id):
            continue

        # get the percent changes for each fips
        test_fips_df = percent_change_yoy_df[percent_change_yoy_df["cust_id"] == search_fips][cols].copy()

        # MSE against original
        
        base_information = base_information.apply(pd.to_numeric, errors='coerce')
        base_information = base_information.reset_index(drop=True)
        
        test_fips_df = test_fips_df.apply(pd.to_numeric, errors='coerce')
        test_fips_df = test_fips_df.reset_index(drop=True)

        # manually calculate the mse
        total_value = 0

        squared_differences = base_information - test_fips_df
        squared_differences = squared_differences ** 2

        # Calculate the Mean Squared Error for each column
        mse_by_column = squared_differences.mean()

        # Calculate the overall Mean Squared Error
        overall_mse = mse_by_column.mean()

        # check if the new mse is less than before
        if overall_mse < max(min_mse_val):

            mse_ind = min_mse_val.index(max(min_mse_val))

            min_mse_fips[mse_ind] = search_fips
            min_mse_val[mse_ind] = overall_mse
    
    # now I should have an array of all of the cities that are closest in terms of percentage growth


    return min_mse_fips, min_mse_val

percent_change_yoy_df

In [ ]:
# in order to make sure that we have enough data, we will augment in the following ways:
# 1) closest 3 cities to this city by lat-lon + their GDP's and GDP growth
# 2) lat and lon of the other cities
# 3) distance to the other cities

# let's start by augmenting our dataframe of stopped routes cities
mse_dictionary = {}

# getting statistics on how often each industry category drops after regional flights stop
for idx, row in temp_shutdown_df.iterrows():

    # get the FIPS
    fips_idx = row["fips_affected"]

    id_dict = {}

    # for each of the fips idx
    for code in fips_idx:

        print(f"ANALYZE: {custom_fips_code(code)}")

        closest_similarity_cities, closest_similarity_errors = get_closest_by_growth(code, row["Year"], first_year = 2001, same_time_frame=True, num_to_find=50)
        id_dict[custom_fips_code(code)] = sorted(list(zip(closest_similarity_cities, closest_similarity_errors)), key=lambda x: x[1])

        print(closest_similarity_cities)
        print(closest_similarity_errors)

    mse_dictionary[row["citadel_id"]] = id_dict 



In [ ]:
# pickle the information just incase anything happens down the line:
# import pickle

import copy

backup_dict = copy.deepcopy(mse_dictionary)


In [ ]:

backup_dict2 = copy.deepcopy(mse_dictionary)

# print(os.getcwd())

# with open("./pickle_files/mse_comps_for_flights_dictionary.pkl", "wb") as file:
#     pickle.dump(mse_dictionary, file)

# open_f = open(f"~/Desktop/mse_comps_for_flights_dictionary.pkl", "wb")
# pickle.dump(mse_dictionary, open_f)



In [ ]:
import pickle

with open(file_path, 'rb') as file:
    mse_dictionary = pickle.load(file)

In [ ]:
# now we are going to prepare dataframes for synthetic control models below

route_and_affected_fips = {}
scm_dataframe_dict = {}

# here is the control flow
# we are going to start by going through each of the keys
# get that data from the percent change 
# go through each of the constituent comparable models
# get their probabilities
# for each of the types of gdp that we consider, we collect all the information
# pass the function below each type of gdp for each of the control and treated regions
# that will create the plots that we need

# mse dict is indexed by route code
# dict in there is indexed by fips code
# that has an array of similar fips and their mse

def get_mse_score(fips_id, dict):

    return dict[fips_id]


# iterate through each of the keys
for route_id in mse_dictionary.keys():

    # get the list of comparables in order
    # dictionary of affected fips id's and their closest parters w mse's
    closest_matches = mse_dictionary[route_id]
    year_affected = temp_shutdown_df.loc[temp_shutdown_df["citadel_id"] == route_id, "Year"].values[-1]

    route_and_affected_fips[route_id] = (closest_matches.keys(), year_affected)
    scm_dataframe_dict[route_id] = {}

    # go through each of the fips that were affected by the route closure
    for affected_fips_code in closest_matches.keys():

        # create a dataframe
        temp_route_scm_df = percent_change_yoy_df[percent_change_yoy_df["cust_id"] == affected_fips_code].copy()
        temp_route_scm_df["mse_score"] = 0

        # now go through each of the ids and get their data
        for (fips_id, mse) in closest_matches[affected_fips_code]:

            temp_route_scm_df = pd.concat([temp_route_scm_df, percent_change_yoy_df[percent_change_yoy_df["cust_id"] == fips_id]])
            temp_route_scm_df.loc[temp_route_scm_df["cust_id"] == fips_id, "mse_score"] = mse

        # temp_route_scm_df["mse_score"] = temp_route_scm_df.apply(lambda row: get_mse_score(row["citadel_id"], closest_matches))
        # temp_route_scm_df["route_deprecated"] = (temp_route_scm_df["citadel_id"] == route_id)
        temp_route_scm_df["affected"] = (temp_route_scm_df["cust_id"] == affected_fips_code)
        scm_dataframe_dict[route_id][affected_fips_code] = temp_route_scm_df.copy()
        

In [ ]:
scm_dataframe_dict[3150332467]['agaad']


In [ ]:

%pip install statsmodels
%pip install pmdarima
%pip install SyntheticControlMethods


In [ ]:
import statsmodels.api as sm
from pmdarima import auto_arima
from SyntheticControlMethods import Synth, DiffSynth
import statsmodels.formula.api as smf
from sklearn.linear_model import Lasso

In [ ]:
# now go through each of the items in the dictionary and create stats models for each of them


# this function creates the synthetic control model and plots the data
# to visualize the effect of route attrition
total_year_cols = [str(yr) for yr in list(range(2001, 2022))]

def synthetic_control_model(pct_chng_df, route_attrition_year, response_gdp, affected_fips_id):

    # we need to transpose the matrix that we are given to start
    # that way the description gdp is one of the columns
    # and the years are the indexes

    temp_cols = ["Description"]
    
    for year_col in total_year_cols:
        temp_cols.append(year_col)



    # get rid of any data that has a changed route
    arr_1 = pct_chng_df[pct_chng_df["cust_id"] == custom_fips_code(affected_fips_id)]
    arr_2 = pct_chng_df[pct_chng_df["route_gone"] == False]

    pct_chng_df = pd.concat([arr_1, arr_2]).reset_index(drop=True)


    # we are going to go index by index, transpose the matrix, and add the other columns back in as necessary
    fips_ids = pct_chng_df["GeoFIPS"].unique()

    # df that we are going to be writing to
    total_input_data = pd.DataFrame(columns=["Year", response_gdp, "GeoFIPS", "cust_id"])

    # go through each of the fips ids and get the data that is associated with them
    for fips_id in fips_ids:

        # get the data that is associated with that fips id
        temp_fips_df = pct_chng_df[pct_chng_df["cust_id"] == custom_fips_code(fips_id)]

        # get the transpose of the matrix
        temp_fips_df = temp_fips_df[temp_cols]
        temp_fips_df = temp_fips_df.T

        # set the new column headers
        new_column_headers = temp_fips_df.iloc[0]
        temp_fips_df = temp_fips_df[1:]


        # Set the new column headers and rename the axis
        temp_fips_df.columns = new_column_headers
        temp_fips_df = temp_fips_df.rename_axis(None, axis=1)

        # rename and get only the interesting data
        temp_fips_df = temp_fips_df[response_gdp].reset_index().rename(columns = {"index" : "Year"})

        # add the ids
        temp_fips_df["GeoFIPS"] = str(fips_id)
        temp_fips_df["cust_id"] = str(custom_fips_code(fips_id))

        total_input_data = pd.concat([total_input_data, temp_fips_df])

        
    # go ahead and adjust the data frame
    total_input_data["Year"] = total_input_data["Year"].apply(pd.to_numeric, errors='coerce')
    total_input_data["target"] = total_input_data["cust_id"] == custom_fips_code(affected_fips_id)
    total_input_data["after_treatment"] = total_input_data["Year"] >= int(route_attrition_year)

    ax = plt.subplot(1, 1, 1)

    (total_input_data
        .assign(target = np.where(total_input_data["target"], "Target", "Other Counties"))
        .groupby(["Year", "target"])[response_gdp]
        .mean()
        .reset_index()
        .pivot(index="Year", columns="target", values=response_gdp)
        .plot(ax=ax, figsize=(10,5)))
    


    inverted = (total_input_data.query("~after_treatment") # filter pre-intervention period
            .pivot(index='cust_id', columns="Year")[response_gdp] # make one column per year and one row per state
            .T) # flip the table to have one column per state

    y = inverted[custom_fips_code(affected_fips_id)].values    
    X = inverted.drop(columns=custom_fips_code(affected_fips_id)).values  # other states 

    weights_lr = Lasso(fit_intercept=False).fit(X, y).coef_.round(3)

    print(weights_lr)

    synthetic = (total_input_data.query("~target")
                  .pivot(index='Year', columns="cust_id")[response_gdp]
                  .values.dot(weights_lr))
    

    return (total_input_data
            .query(f"cust_id=={custom_fips_code(affected_fips_id)}")[["cust_id", "Year", response_gdp, "after_treatment"]]
            .assign(synthetic=synthetic))





def avg_SCM(pct_chng_df, route_attrition_year, response_gdp, affected_fips_id, pre, post, quarter):

    # we need to transpose the matrix that we are given to start
    # that way the description gdp is one of the columns
    # and the years are the indexes

    temp_cols = ["Description"]
    
    for year_col in total_year_cols:
        temp_cols.append(year_col)



    # get rid of any data that has a changed route
    arr_1 = pct_chng_df[pct_chng_df["cust_id"] == custom_fips_code(affected_fips_id)]
    arr_2 = pct_chng_df[pct_chng_df["route_gone"] == False]

    pct_chng_df = pd.concat([arr_1, arr_2]).reset_index(drop=True)


    # we are going to go index by index, transpose the matrix, and add the other columns back in as necessary
    fips_ids = pct_chng_df["GeoFIPS"].unique()

    # df that we are going to be writing to
    total_input_data = pd.DataFrame(columns=["Year", response_gdp, "GeoFIPS", "cust_id"])

    # go through each of the fips ids and get the data that is associated with them
    for fips_id in fips_ids:

        # get the data that is associated with that fips id
        temp_fips_df = pct_chng_df[pct_chng_df["cust_id"] == custom_fips_code(fips_id)]

        # get the transpose of the matrix
        temp_fips_df = temp_fips_df[temp_cols]
        temp_fips_df = temp_fips_df.T

        # set the new column headers
        new_column_headers = temp_fips_df.iloc[0]
        temp_fips_df = temp_fips_df[1:]


        # Set the new column headers and rename the axis
        temp_fips_df.columns = new_column_headers
        temp_fips_df = temp_fips_df.rename_axis(None, axis=1)

        # rename and get only the interesting data
        temp_fips_df = temp_fips_df[response_gdp].reset_index().rename(columns = {"index" : "Year"})

        # add the ids
        temp_fips_df["GeoFIPS"] = str(fips_id)
        temp_fips_df["cust_id"] = str(custom_fips_code(fips_id))

        total_input_data = pd.concat([total_input_data, temp_fips_df])

        
    # go ahead and adjust the data frame
    total_input_data["Year"] = total_input_data["Year"].apply(pd.to_numeric, errors='coerce')
    total_input_data["target"] = total_input_data["cust_id"] == custom_fips_code(affected_fips_id)
    total_input_data["after_treatment"] = total_input_data["Year"] >= int(route_attrition_year)

    # plt.figure(figsize=(10,6))
    # plt.plot(total_input_data.query("target")["Year"], total_input_data.query("target")[response_gdp], label="Target")
    # plt.plot(total_input_data.query("target")["Year"], routes_synth_lr, label="Synthetic Control")
    # plt.vlines(x=route_attrition_year, ymin=-1, ymax=1, linestyle=":", lw=2, label="Route Attrition")
    # plt.ylabel("GDP Growth Over Years")
    # plt.legend()

    # sc = Synth(total_input_data, response_gdp, "GeoFIPS", "Year", route_attrition_year, affected_fips_id, n_optim=100)
    # sc.plot(["original", "pointwise", "cumulative"], treated_label=affected_fips_id, synth_label="Synthetic County", treatment_label="Route Closure")


    treated_df = total_input_data[total_input_data['cust_id'] == custom_fips_code(affected_fips_id)]
    control_df = total_input_data[~(total_input_data['cust_id'] == custom_fips_code(affected_fips_id))]


    # # Separate pre- and post-policy data
    # pre_policy = total_input_data[total_input_data['Year'] < int(route_attrition_year)]
    # post_policy = total_input_data[total_input_data['Year'] >= int(route_attrition_year)]


    # Fit ARIMA model for each control region
    # # control_arima_models = {}
    # print(F"AFFECTED REGIONS: {affected_fips_id}")
    # print(F"CONTROL REGIONS: {list(set(fips_ids))}")


    # # go through each of the data sets
    # for untouched_id in list(set(fips_ids)):

    #     if untouched_id == affected_fips_id:
    #         continue

    #     # get untouched regions
    #     region_df = control_df[control_df['cust_id'] == custom_fips_code(untouched_id)]

    #     # gridsearch for the model
    #     control_arima_models[untouched_id] = auto_arima(region_df[response_gdp], seasonal=False)


    # # Fit ARIMA model for the treated region
    # treated_arima_model = auto_arima(treated_df[response_gdp], seasonal=False)


    # # Predict using ARIMA models for control regions
    # control_predictions = {}
    # for region, model in control_arima_models.items():

    #     control_predictions[region] = model.predict(len(pre_policy) + len(post_policy))


    # # Create synthetic control group by combining predictions from control regions
    # synthetic_control_group = np.mean(list(control_predictions.values()), axis=0)

    # print(f"SYNTHETIC LENGTH: {len(synthetic_control_group)}")
    # print(f"{response_gdp}: {len(control_predictions)}")

    # data_pairs = zip(pre_policy['Year'], synthetic_control_group[:len(pre_policy)])

    # for p in data_pairs:
    #     print(p)


    # # Plot post-policy data
    # plt.figure(figsize=(10, 6))
    # plt.plot(pre_policy['Year'], treated_arima_model.predict(len(pre_policy)), label=f'{response_gdp} (Pre-policy)')
    # plt.plot(pre_policy['Year'], synthetic_control_group[:len(pre_policy)], label='Synthetic Control (Pre-policy)')
    # plt.plot(post_policy['Year'], treated_arima_model.predict(len(post_policy)), label=f'{"Affected County"} (Treated)')
    # plt.plot(post_policy['Year'], synthetic_control_group[len(pre_policy):], label='Synthetic Control')
    # # plt.plot(pre_policy['Year'], pre_policy[response_gdp], 'o', label='Actual Data (Pre-policy)')
    # # plt.plot(post_policy['Year'], post_policy[response_gdp], 'o', label='Actual Data (Post-policy)')
    # plt.axvline(route_attrition_year, color='red', linestyle='--', label='Policy Change Date')
    # plt.xlabel('Year')
    # plt.ylabel(response_gdp)
    # plt.title('Effect of Policy Change on Local Economies')
    # plt.legend()
    # plt.show()
    # Plot post-policy data


    # get the weighted averages of the others:

    try:

        name = fips_mapping_df[fips_mapping_df["fips_code"].apply(custom_fips_code) == custom_fips_code(affected_fips_id)]["name"].values[0]
    
    except:

        name = custom_fips_code(affected_fips_id)


    # plt.figure(figsize=(10, 6))
    # plt.plot(treated_df['Year'][route_attrition_year - 2001 -  pre : route_attrition_year - 2001 + post], treated_df[response_gdp][route_attrition_year - 2001 -  pre : route_attrition_year - 2001 + post], label=f'{response_gdp} (Treated)')
    # plt.plot(treated_df['Year'][route_attrition_year - 2001 -  pre : route_attrition_year - 2001 + post], pd.concat([total_input_data.groupby('Year')[response_gdp].mean()[route_attrition_year - 2001 -  pre : route_attrition_year - 2001], control_df.groupby('Year')[response_gdp].mean()[route_attrition_year - 2001 : route_attrition_year - 2001 + post]]), label='Synthetic Control')
    # # plt.plot(post_policy['Year'], treated_arima_model.predict(len(post_policy)), label=f'{"Affected County"} (Treated)')
    # # plt.plot(post_policy['Year'], synthetic_control_group[len(pre_policy):], label='Synthetic Control')
    # # plt.plot(pre_policy['Year'], pre_policy[response_gdp], 'o', label='Actual Data (Pre-policy)')
    # # plt.plot(post_policy['Year'], post_policy[response_gdp], 'o', label='Actual Data (Post-policy)')
    # plt.axvline(route_attrition_year + (quarter/4) - (0.125), color='red', linestyle='--', label='Route Closure')
    # plt.xlabel('Year')
    # plt.ylabel(response_gdp)
    # plt.title(f'Effect of Route Closure on {name} {response_gdp}')
    # plt.legend()
    # plt.show()


    # print(fips_mapping_df[fips_mapping_df["mkt_id"] == affected_fips_id]["name"])
    
    
    # print(fips_mapping_df.loc[fips_mapping_df["mkt_id"] == affected_fips_id, "name"].values[0])
    
    # # Plot results
    # plt.figure(figsize=(10, 6))
    # plt.plot(post_policy['Year'], treated_arima_model.predict(len(post_policy)), label=f'{response_gdp} (Post-policy)')
    # plt.plot(post_policy['Year'], synthetic_control_group[len(post_policy):], label='Synthetic Control (Post-policy)')
    # plt.plot(post_policy['Year'], post_policy[response_gdp], 'o', label='Actual Data (Post-policy)')
    # plt.axvline(route_attrition_year, color='red', linestyle='--', label='Policy Change Date')
    # plt.xlabel('Year')
    # plt.ylabel(response_gdp)
    # plt.title('Effect of Policy Change on Local Economies')
    # plt.legend()
    # plt.show()

    # # PlYear


# # Example usage:
# # synthetic_control_model(df, treated_region='TreatedRegion', control_regions=['ControlRegion1', 'ControlRegion2'],
# #                         policy_change_date='2023-01-01', response_variable='GDP')


In [ ]:
# # # getting the treated region

# route_ids = scm_dataframe_dict.keys()

# for route in route_ids:

#     for county_code in scm_dataframe_dict[route].keys():

#         # for desc in ["Utilities", "Agriculture, forestry, fishing and hunting", "Administrative and support and waste management and remediation services", "Professional, scientific, and technical services", "Private goods-producing industries 2/", "Durable goods manufacturing", "Finance and insurance", "Management of companies and enterprises", "Educational services", "Manufacturing", "Professional and business services", "Arts, entertainment, and recreation", "Mining, quarrying, and oil and gas extraction"]:
#         # for desc in ["Utilities", "Agriculture, forestry, fishing and hunting", "Administrative and support and waste management and remediation services", "Professional, scientific, and technical services", "Private goods-producing industries 2/", "Durable goods manufacturing"]:
#         for desc in ["Utilities", "Management of companies and enterprises", "Manufacturing", "Professional and business services", "Arts, entertainment, and recreation", "Durable goods manufacturing"]:

#             try:
                
#                 quarter = temp_shutdown_df[temp_shutdown_df["citadel_id"] == route]["quarter"].values[-1]

#                 affected_fips_id = scm_dataframe_dict[route][county_code].loc[scm_dataframe_dict[route][county_code]["affected"] == True, "GeoFIPS"].values[0]
#                 # print(f"ID WE ARE CONSIDERING: {affected_fips_id}")
#                 if route_and_affected_fips[route][1] != 2008 and route_and_affected_fips[route][1] != 2009:
                    
#                     avg_SCM(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], desc, affected_fips_id, 5, 2, quarter)
#                     avg_SCM(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], desc, affected_fips_id, 5, 3, quarter)
#                     avg_SCM(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], desc, affected_fips_id, 5, 4, quarter)
                
#             except:

#                 pass


# # control_pool = scm_dataframe_dict[3150332467]["agaad"]["GeoFIPS"].unique()

# # dataset = synthetic_control_model(scm_dataframe_dict[3150332467]["agaad"], route_and_affected_fips[3150332467][1],  "All industry total", affected_fips_id)

# # calif_synth_lr = (dataset.query("~target")
# #                   .pivot(index='Year', columns="cust_id")["All industry total"]
# #                   .values.dot(weights_lr))

# # sinthetic_states = [synthetic_control_model(scm_dataframe_dict[3150332467]["agaad"], route_and_affected_fips[3150332467][1], "All industry total", control_id) for control_id in control_pool]

# # plt.figure(figsize=(12,7))
# # for state in sinthetic_states:
# #     plt.plot(state["Year"], state["All industry total"] - state["synthetic"], color="C5",alpha=0.4)

# # plt.plot(scm_dataframe_dict[3150332467]["agaad"].query("target")["Year"], scm_dataframe_dict[3150332467]["agaad"].query("california")["cigsale"] - calif_synth_lr,
# #         label="California");

# # plt.vlines(x=1988, ymin=-50, ymax=120, linestyle=":", lw=2, label="Proposition 99")
# # plt.hlines(y=0, xmin=1970, xmax=2000, lw=3)
# # plt.ylabel("Gap in per-capita cigarette sales (in packs)")
# # plt.title("State - Synthetic Across Time")
# # plt.legend()

# # # , route_attrition_year, response_gdp



In [ ]:

%pip install dill

import dill
dill.dump_session('/Users/tristanbrigham/Desktop/notebook_env.db')

In [ ]:
# print(pct_chng_df)
import dill
dill.load_session('/Users/tristanbrigham/Desktop/notebook_env.db')


In [ ]:


# control_pool = scm_dataframe_dict[3150332467]["agaad"]["GeoFIPS"].unique()

# dataset = synthetic_control_model(scm_dataframe_dict[3150332467]["agaad"], route_and_affected_fips[3150332467][1],  "All industry total", affected_fips_id)

# calif_synth_lr = (dataset.query("~target")
#                   .pivot(index='Year', columns="cust_id")["All industry total"]
#                   .values.dot(weights_lr))

# sinthetic_states = [synthetic_control_model(scm_dataframe_dict[3150332467]["agaad"], route_and_affected_fips[3150332467][1], "All industry total", control_id) for control_id in control_pool]

# plt.figure(figsize=(12,7))
# for state in sinthetic_states:
#     plt.plot(state["Year"], state["All industry total"] - state["synthetic"], color="C5",alpha=0.4)

# plt.plot(scm_dataframe_dict[3150332467]["agaad"].query("target")["Year"], scm_dataframe_dict[3150332467]["agaad"].query("california")["cigsale"] - calif_synth_lr,
#         label="California");

# plt.vlines(x=1988, ymin=-50, ymax=120, linestyle=":", lw=2, label="Proposition 99")
# plt.hlines(y=0, xmin=1970, xmax=2000, lw=3)
# plt.ylabel("Gap in per-capita cigarette sales (in packs)")
# plt.title("State - Synthetic Across Time")
# plt.legend()

# # , route_attrition_year, response_gdp
import statsmodels.api as sm
def new_avg_SCM(pct_chng_df, route_attrition_year, response_gdp, affected_fips_id, pre, post, quarter):

    # we need to transpose the matrix that we are given to start
    # that way the description gdp is one of the columns
    # and the years are the indexes

    temp_cols = ["Description"]
    
    for year_col in total_year_cols:
        temp_cols.append(year_col)



    # get rid of any data that has a changed route
    arr_1 = pct_chng_df[pct_chng_df["cust_id"] == custom_fips_code(affected_fips_id)]
    arr_2 = pct_chng_df[pct_chng_df["route_gone"] == False]

    pct_chng_df = pd.concat([arr_1, arr_2]).reset_index(drop=True)

    minimum_value = pct_chng_df[pct_chng_df["GeoFIPS"] != affected_fips_id]['mse_score'].min()
    min_fips = pct_chng_df[pct_chng_df["mse_score"] == minimum_value]['GeoFIPS'].values[0]


    # we are going to go index by index, transpose the matrix, and add the other columns back in as necessary
    fips_ids = pct_chng_df["GeoFIPS"].unique()
    # fips_ids = [affected_fips_id, min_fips]

    # df that we are going to be writing to
    total_input_data = pd.DataFrame(columns=["Year", response_gdp, "GeoFIPS", "cust_id"])

    # go through each of the fips ids and get the data that is associated with them
    for fips_id in fips_ids:

        # get the data that is associated with that fips id
        temp_fips_df = pct_chng_df[pct_chng_df["cust_id"] == custom_fips_code(fips_id)]

        # get the transpose of the matrix
        temp_fips_df = temp_fips_df[temp_cols]
        temp_fips_df = temp_fips_df.T

        # set the new column headers
        new_column_headers = temp_fips_df.iloc[0]
        temp_fips_df = temp_fips_df[1:]


        # Set the new column headers and rename the axis
        temp_fips_df.columns = new_column_headers
        temp_fips_df = temp_fips_df.rename_axis(None, axis=1)

        # rename and get only the interesting data
        temp_fips_df = temp_fips_df[response_gdp].reset_index().rename(columns = {"index" : "Year"})

        # add the ids
        temp_fips_df["GeoFIPS"] = str(fips_id)
        temp_fips_df["cust_id"] = str(custom_fips_code(fips_id))

        total_input_data = pd.concat([total_input_data, temp_fips_df])

        
    # go ahead and adjust the data frame
    total_input_data["Year"] = total_input_data["Year"].apply(pd.to_numeric, errors='coerce')
    total_input_data["target"] = total_input_data["cust_id"] == custom_fips_code(affected_fips_id)
    total_input_data["after_treatment"] = total_input_data["Year"] >= int(route_attrition_year)

    # plt.figure(figsize=(10,6))
    # plt.plot(total_input_data.query("target")["Year"], total_input_data.query("target")[response_gdp], label="Target")
    # plt.plot(total_input_data.query("target")["Year"], routes_synth_lr, label="Synthetic Control")
    # plt.vlines(x=route_attrition_year, ymin=-1, ymax=1, linestyle=":", lw=2, label="Route Attrition")
    # plt.ylabel("GDP Growth Over Years")
    # plt.legend()

    sc = Synth(total_input_data, response_gdp, "GeoFIPS", "Year", route_attrition_year, affected_fips_id, n_optim=100)
    sc.plot(["original", "pointwise", "cumulative"], treated_label=affected_fips_id, synth_label="Synthetic County", treatment_label="Route Closure")


    treated_df = total_input_data[total_input_data['cust_id'] == custom_fips_code(affected_fips_id)]
    control_df = total_input_data[~(total_input_data['cust_id'] == custom_fips_code(affected_fips_id))]


    # Separate pre- and post-policy data
    pre_policy = total_input_data[total_input_data['Year'] < int(route_attrition_year)]
    post_policy = total_input_data[total_input_data['Year'] >= int(route_attrition_year)]


    Fit ARIMA model for each control region
    # control_arima_models = {}
    print(F"AFFECTED REGIONS: {affected_fips_id}")
    print(F"CONTROL REGIONS: {list(set(fips_ids))}")


    # go through each of the data sets
    for untouched_id in list(set(fips_ids)):

        if untouched_id == affected_fips_id:
            continue

        # get untouched regions
        region_df = control_df[control_df['cust_id'] == custom_fips_code(untouched_id)]

        # gridsearch for the model
        control_arima_models[untouched_id] = auto_arima(region_df[response_gdp], seasonal=False)


    # Fit ARIMA model for the treated region
    treated_arima_model = auto_arima(treated_df[response_gdp], seasonal=False)


    # Predict using ARIMA models for control regions
    control_predictions = {}
    for region, model in control_arima_models.items():

        control_predictions[region] = model.predict(len(pre_policy) + len(post_policy))


    # Create synthetic control group by combining predictions from control regions
    synthetic_control_group = np.mean(list(control_predictions.values()), axis=0)

    print(f"SYNTHETIC LENGTH: {len(synthetic_control_group)}")
    print(f"{response_gdp}: {len(control_predictions)}")

    data_pairs = zip(pre_policy['Year'], synthetic_control_group[:len(pre_policy)])

    for p in data_pairs:
        print(p)


    # # Plot post-policy data
    # plt.figure(figsize=(10, 6))
    # plt.plot(pre_policy['Year'], treated_arima_model.predict(len(pre_policy)), label=f'{response_gdp} (Pre-policy)')
    # plt.plot(pre_policy['Year'], synthetic_control_group[:len(pre_policy)], label='Synthetic Control (Pre-policy)')
    # plt.plot(post_policy['Year'], treated_arima_model.predict(len(post_policy)), label=f'{"Affected County"} (Treated)')
    # plt.plot(post_policy['Year'], synthetic_control_group[len(pre_policy):], label='Synthetic Control')
    # # plt.plot(pre_policy['Year'], pre_policy[response_gdp], 'o', label='Actual Data (Pre-policy)')
    # # plt.plot(post_policy['Year'], post_policy[response_gdp], 'o', label='Actual Data (Post-policy)')
    # plt.axvline(route_attrition_year, color='red', linestyle='--', label='Policy Change Date')
    # plt.xlabel('Year')
    # plt.ylabel(response_gdp)
    # plt.title('Effect of Policy Change on Local Economies')
    # plt.legend()
    # plt.show()
    # Plot post-policy data

    treated_df['Year'][route_attrition_year - 2001 -  pre : route_attrition_year - 2001 + post], treated_df[response_gdp][route_attrition_year - 2001 -  pre : route_attrition_year - 2001 + post]
    
    df_after_years = treated_df['Year'][route_attrition_year - 2001 -  pre : route_attrition_year - 2001]
    df_before_years = treated_df['Year'][route_attrition_year - 2001 -  pre : route_attrition_year - 2001]

    # df_before_data_treated = pd.DataFrame(treated_df[response_gdp][route_attrition_year - 2001 -  pre : route_attrition_year - 2001], columns=["gdp"])
    df_before_data_treated = treated_df[response_gdp][route_attrition_year - 2001 -  pre : route_attrition_year - 2001].to_frame()
    df_before_data_treated['t'] = 0
    df_before_data_treated['g'] = 1
    df_before_data_treated.rename(columns={response_gdp : "gdp"}, inplace=True)
    df_before_data_treated = df_before_data_treated.reset_index(drop=True)
    

    # df_after_data_treated = pd.DataFrame(treated_df[response_gdp][route_attrition_year - 2001 -  pre : route_attrition_year - 2001], columns=["gdp"])
    df_after_data_treated = treated_df[response_gdp][route_attrition_year - 2001 -  pre : route_attrition_year - 2001].to_frame()
    df_after_data_treated['t'] = 1
    df_after_data_treated['g'] = 1
    df_after_data_treated.rename(columns={response_gdp : "gdp"}, inplace=True)
    df_after_data_treated = df_after_data_treated.reset_index(drop=True)

    # df_before_normal = pd.DataFrame(total_input_data.groupby('Year')[response_gdp].mean()[route_attrition_year - 2001 -  pre : route_attrition_year - 2001], columns=["gdp"])
    df_before_normal = total_input_data.groupby('Year')[response_gdp].mean()[route_attrition_year - 2001 -  pre : route_attrition_year - 2001].to_frame()
    df_before_normal['t'] = 0
    df_before_normal['g'] = 0
    df_before_normal.rename(columns={response_gdp : "gdp"}, inplace=True)
    df_before_normal = df_before_normal.reset_index(drop=True)

    # df_after_normal = pd.DataFrame(control_df.groupby('Year')[response_gdp].mean()[route_attrition_year - 2001 : route_attrition_year - 2001 + post], columns=["gdp"])
    df_after_normal = control_df.groupby('Year')[response_gdp].mean()[route_attrition_year - 2001 : route_attrition_year - 2001 + post].to_frame()
    df_after_normal['t'] = 1
    df_after_normal['g'] = 0
    df_after_normal.rename(columns={response_gdp : "gdp"}, inplace=True)
    df_after_normal = df_after_normal.reset_index(drop=True)



    # total_dat = pd.DataFrame(columns=['gdp', 't', 'g'])

    total_dat = df_before_data_treated
    total_dat = pd.concat([total_dat, df_after_data_treated])
    total_dat = pd.concat([total_dat, df_before_normal])
    total_dat = pd.concat([total_dat, df_after_normal])
    # print(total_dat)

    # create the interaction 
    total_dat['gt'] = total_dat.g * total_dat.t

    total_dat[['g', 't', 'gt']] = total_dat[['g', 't', 'gt']].astype(float)
    total_dat['gdp'] = pd.to_numeric(total_dat['gdp'], errors='coerce')
    total_dat = total_dat.dropna(subset=['gdp'])


    from sklearn.linear_model import LinearRegression
    lr = LinearRegression()

    X = total_dat[['g', 't', 'gt']]
    y = total_dat['gdp']

    lr.fit(X, y)
    # print(lr.coef_)  # the coefficient for gt is the DID, which is 2.75

    from statsmodels.formula.api import ols
    ols = sm.OLS(y, sm.add_constant(X)).fit()
    # ols = ols('gdp ~ g + t + gt', data=total_dat).fit()
    # print(ols.summary())
    
    
    old_p_val = ols.pvalues['gt']
    impact_score = lr.coef_[2]

    if old_p_val < 0.2:

        try:

            name = fips_mapping_df[fips_mapping_df["fips_code"].apply(custom_fips_code) == custom_fips_code(affected_fips_id)]["name"].values[0]
        
        except:

            name = custom_fips_code(affected_fips_id)

        print()
        print()
        print()
        print()
        print()
        print()
        print()
        print()
        print()
        print()
        print()
        print()
        print(ols.summary())
        print(f"P_VAL: {old_p_val}")

        # plt.figure(figsize=(10, 6))
        # plt.plot(treated_df['Year'][route_attrition_year - 2001 -  pre : route_attrition_year - 2001 + post], treated_df[response_gdp][route_attrition_year - 2001 -  pre : route_attrition_year - 2001 + post], label=f'{response_gdp} (Treated)')
        # plt.plot(treated_df['Year'][route_attrition_year - 2001 -  pre : route_attrition_year - 2001 + post], pd.concat([total_input_data.groupby('Year')[response_gdp].mean()[route_attrition_year - 2001 -  pre : route_attrition_year - 2001], control_df.groupby('Year')[response_gdp].mean()[route_attrition_year - 2001 : route_attrition_year - 2001 + post]]), label='Synthetic Control')
        # # plt.plot(post_policy['Year'], treated_arima_model.predict(len(post_policy)), label=f'{"Affected County"} (Treated)')
        # # plt.plot(post_policy['Year'], synthetic_control_group[len(pre_policy):], label='Synthetic Control')
        # # plt.plot(pre_policy['Year'], pre_policy[response_gdp], 'o', label='Actual Data (Pre-policy)')
        # # plt.plot(post_policy['Year'], post_policy[response_gdp], 'o', label='Actual Data (Post-policy)')
        # plt.axvline(route_attrition_year + (quarter/4) - (0.125), color='red', linestyle='--', label='Route Closure')
        # plt.xlabel('Year')
        # plt.ylabel(response_gdp)
        # plt.title(f'Effect of Route Closure on {name} {response_gdp}')
        # plt.legend()
        # plt.show()

    return old_p_val, impact_score


# # getting the treated region

route_ids = scm_dataframe_dict.keys()


for desc in ["Utilities", "Agriculture, forestry, fishing and hunting", "Administrative and support and waste management and remediation services", "Professional, scientific, and technical services", "Private goods-producing industries 2/", "Durable goods manufacturing", "Finance and insurance", "Management of companies and enterprises", "Educational services", "Manufacturing", "Professional and business services", "Arts, entertainment, and recreation", "Mining, quarrying, and oil and gas extraction"]:
# for desc in ["Utilities", "Agriculture, forestry, fishing and hunting", "Administrative and support and waste management and remediation services", "Professional, scientific, and technical services", "Private goods-producing industries 2/", "Durable goods manufacturing"]:
# for desc in ["Utilities", "Management of companies and enterprises", "Manufacturing", "Professional and business services", "Arts, entertainment, and recreation", "Durable goods manufacturing"]:
    
    
    for route in route_ids:

        for county_code in scm_dataframe_dict[route].keys():

            try:
                
                affected_fips_id = scm_dataframe_dict[route][county_code].loc[scm_dataframe_dict[route][county_code]["affected"] == True, "GeoFIPS"].values[0]
                quarter = temp_shutdown_df[temp_shutdown_df["citadel_id"] == route]["quarter"].values[-1]

                name = fips_mapping_df[fips_mapping_df["fips_code"].apply(custom_fips_code) == custom_fips_code(affected_fips_id)]["name"].values[0]

                # if name == "Amador" and quarter == 1 and route_and_affected_fips[route][1] == 2016:
                    # synthetic_control_model(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], desc, affected_fips_id)

                for horizon in range(2, 4):
                    
                    p_val, did = new_avg_SCM(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], desc, affected_fips_id, 5, horizon, quarter)

                    if p_val < 0.2 and route_and_affected_fips[route][1] != 2009:

                        print(f"{route} | {desc} | {horizon} | {did} | {p_val} | {name} | {quarter} | {affected_fips_id} | {route_and_affected_fips[route][1]}")

                        # avg_SCM(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], desc, affected_fips_id, 5, 2, quarter)

            # if name == "Tuolumne" and quarter == 4 and route_and_affected_fips[route][1] == 2006:
            #     print()
            #     print()
            #     print()
            #     print(f"Arts, entertainment, and recreation: {route}")
            #     # synthetic_control_model(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], desc, affected_fips_id)
            #     new_avg_SCM(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], "Arts, entertainment, and recreation", affected_fips_id, 5, 2, quarter)
            #     print(route)
            #     print()
            #     print()

            #     break

                



            # # print(f"ID WE ARE CONSIDERING: {affected_fips_id}")
            # if route_and_affected_fips[route][1] != 2008 and route_and_affected_fips[route][1] != 2009:
                
            #     avg_SCM(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], desc, affected_fips_id, 5, 2, quarter)
            #     avg_SCM(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], desc, affected_fips_id, 5, 3, quarter)
            #     avg_SCM(scm_dataframe_dict[route][county_code], route_and_affected_fips[route][1], desc, affected_fips_id, 5, 4, quarter)

                
            except:

                pass






# Amount and Type of Delays Leading Up to Airport Closure

looking at year vs. ratio closures

# Amount and Type of Delays Leading Up to Airport Closure

looking at year vs. ratio closures

### There is no correlation 

# Economic Growth over years vs. Exit of Airlines

# Which Airlines Cut Back on Service After Storms
